In [2]:
import numpy as np
import pandas as pd

# Загрузка тренировочных данных

In [3]:
data_train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=0, names=['text', 'label'])

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [5]:
texts = list(data_train['text'].to_numpy())

In [6]:
labels = list(data_train['label'].to_numpy())

## Подготовим токенизатор и переведем текста в матричное представление

In [15]:
vec = TfidfVectorizer(texts)

In [16]:
X_train = vec.fit_transform(texts)

In [17]:
X_train.shape

(1999, 3973)

# Выбор классификатора

In [19]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(cross_val_score(clf(max_iter=1000), X_train, labels).mean())
    print("\n")

0.7703922305764411


0.7673784461152883


0.7543684210526316




# Добавим биграммы в представление

In [39]:
vec2 = TfidfVectorizer(ngram_range=(3,12), analyzer='char_wb')

In [40]:
X_train2 = vec2.fit_transform(texts)

In [41]:
X_train2.shape

(1999, 64098)

Снова проверяем классификаторы

In [42]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(clf(max_iter=10000), X_train2, labels).mean())

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.7703784461152882
<class 'sklearn.svm._classes.LinearSVC'>
0.7738809523809523
<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.7613646616541352


За основной классификатор выбираю LinearSVC

In [43]:
clf = LogisticRegression().fit(X_train2, labels)

In [44]:
from joblib import dump, load

In [46]:
dump(clf, '../week4/simple_demo/clf.joblib')

['../week4/simple_demo/clf.joblib']

In [15]:
data_test = pd.read_csv('products_sentiment_test.tsv', sep='\t', header=0, names=['ids', 'texts'])

In [16]:
data_test.head()

ids                                              texts
0    0  so , why the small digital elph , rather than ...
1    1  3/4 way through the first disk we played on it...
2    2  better for the zen micro is outlook compatibil...
3    3    6 . play gameboy color games on it with goboy .
4    4  likewise , i 've heard norton 2004 professiona...

In [17]:
X_text_test = list(data_test['texts'].to_numpy())

In [18]:
X_test = vec2.transform(X_text_test)

In [19]:
X_test.shape

(500, 22781)

In [20]:
preds = clf.predict(X_test)

In [22]:
X_text_test[0]

"so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around ."

In [21]:
preds[0]

1

In [121]:
ans = pd.DataFrame({'Id' : np.arange(0, len(preds)), 'y' : np.array(preds)})

In [123]:
ans.to_csv('submission.csv', index=False)

In [50]:
texts[2], labels[2]

('i dont especially like how music files are unstructured ; basically they are just dumped into one folder with no organization , like you might have in windows explorer folders and subfolders .',
 0)